# LAB1: Hello World per Reti Neurali

Consideriamo una successione di valori di X e i corrispondenti valori di Y calcolati mediante una funzione che non conosciamo:

X = -1,  0, 1, 2, 3, 4
Y = -3, -1, 1, 3, 5, 7

Questo è il nostro "dataset".

### SCRIVIAMO UN ALGORITMO TRADIZIONALE (Turing Machine) 
Guardando i valori potremmo pensare che Y = 2X - 1 e scrivere un algoritmo per calcolare Y a partire da un valore qualunque di X:

```
def DueIcsMenoUno(x):
    y = (2 * x) - 1
    return y
```

### PROVIAMO CON LE RETI NEURALI
Ma come possiamo addestrare una rete neurale a svolgere lo stesso compito? 
Vogliamo progettare una rete neurale che possa approssimare la funzione che calcola i valori di Y a partire dai valori di X, senza conoscerla a priori, semplicemente "guardando" i dati.

## Imports


Iniziamo con le importazioni. Qui importiamo [TensorFlow](https://www.tensorflow.org/) e lo chiamiamo `tf` per semplicità.

Il framework che useremo per costruire una rete neurale come sequenza di strati di neuroni si chiama [`keras`](https://keras.io/) ed è contenuto all'interno di TensorFlow, quindi puoi accedervi usando `tf.keras`.

Poi importiamo una libreria chiamata [`numpy`](https://numpy.org) che aiuta a trattare facilmente i dati come array e ad ottimizzare le operazioni numeriche.

In [1]:
import tensorflow as tf
import numpy as np

## Generiamo il dataset

Prima di creare la rete neurale generiamo i dati sui quali lavorerà.
Generiamo due array di 6 elementi, uno per i valori di X, uno per i valori di Y.
La relazione tra X e Y è proprio `y=2x-1`.

In sostanza stiamo creando gli input e gli output per addestrare la rete neurale (fase di `training`).

Il modo più frequente per generare dati (pattern) che verranno dati in input al modello sarà tramite `numpy`, una libreria Python che fornisce diversi tipi di array.
Possiamo creare un array usando [`np.array()`](https://numpy.org/doc/stable/reference/generated/numpy.array.html).

TensorFlow mette a disposizione altre funzioni per lavorare con i dati di input e le vedremo nei prossimi laboratori.

In [2]:
# Creazione degli input e degli output per il modello per l'addestramento.
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Definiamo e compiliamo la rete neurale

Ora costruiamo la rete neurale più semplice possibile utilizzando un solo strato (layer) e un solo neurone.
Utilizziamo la classe [Sequential](https://keras.io/api/models/sequential/) di Keras che consente di creare una rete neurale come sequenza di strati detti [layers](https://keras.io/api/layers/).
Possiamo usare un singolo strato con la classe  [Dense](https://keras.io/api/layers/core_layers/dense/) per costruire questa semplice rete, passando il parametro `units` con valore 1.

È buona pratica definire la "forma" dell'input (shape) per il modello utilizzando `shape` che è un parametro dell'oggetto [tf.keras.Input()](https://www.tensorflow.org/api_docs/python/tf/keras/Input).
In questo caso ogni elemento in xs è uno scalare che possiamo trattare come un vettore monodimensionale perciò `shape` assumerà il valore (1,).

In [3]:
# Creo un semplice modello a strati con Sequential
model = tf.keras.Sequential([

    # Definisco la "forma" dell'input (shape)
    tf.keras.Input(shape=(1,)),

    # Aggiungo uno solo strato
    tf.keras.layers.Dense(units=1)
    ])

Ora possiamo compilare la nostra rete neurale.
Un modello neurale MLP necessità di una [loss function](https://keras.io/api/losses/), o funzione obiettivo, e di un [optimizer](https://keras.io/api/optimizers/) per poter migliorare le prestazioni automaticamente ad ogni epoca sfruttando l'algoritmo backpropagation.

Nel machine learning si utilizza molta matematica che, con a TensorFlow, è stata incapsulata in classi e funzioni facili da usare.

Come funziona questa piccola rete neurale? Abbiamo già descritto la relazione tra Y e X nelle due successioni di numeri che useremo per addestrarla: `y=2x-1`.
La rete però "non conosce" questa relazione e cerca di indovinarla, ad esempio provando `y=10x+10` su tutti i valori di X. La funzione obiettivo `loss` confronterà i valori di Y ottenuti applicando questa funzione "indovinata" agli input rispetto ai valori di Y effettivi e misurerà quanto si discostano in positivo o in negativo.

IL tentativo successivo di indovinare la relazione che lega X e Y viene calcolato utilizzando l'`optimizer` che cercherà di minimizzare il valore della `loss`.
Il secondo tentativo potrebbe essere fatto, ad esempio, utilizzando `y=5x+5` che, pur essendo sbagliato, si avvicina di più alla funzione che vogliamo trovare `y=2x+1` e ci dà un valore di `loss` più basso.

Questo algoritmo viene ripetuto per il numero di _epoche_ che imposteremo nel blocco di codice successivo.

Per specificare quale funzione `loss` e quale `optimizer` vogliamo usare, dobbiamo utilizzare i relativi parametri nel metodo `compile`.
In questo lab abbiamo scelto: 
* `loss`: MSE (Mean Squared Error) o errore quadratico medio: [mean squared error](https://keras.io/api/losses/regression_losses/#meansquarederror-function)
* `optimizer`: SGC (Stocastic Gradient Descent) o gradiente discendente: [stochastic gradient descent](https://keras.io/api/optimizers/sgd/)

Esistono diverse funzioni `loss` e diversi `optimizer` da usare a seconda delle esigenze.


In [4]:
# Compile the model
model.compile(optimizer='sgd', loss='mean_squared_error')

# Addestriamo la Rete Neurale (fase di Training)

Nella fase di training la rete neurale apprende la relazione tra le `x` e le `y`.
Il training si avvia con il metodo [`model.fit()`](https://keras.io/api/models/model_training_apis/#fit-method). 
La fase di addestramento è costituita da un ciclo che si ripete per il numero di _epoche_ impostato durante il quale la rete: 
* Fa un tentativo (`guess`) di indovinare la relazione tra `x` e `y`
* Misura quanto la "bontà" del risultato tramite la `loss` (funzione obiettivo)
* Usa l'`optimizer` per fare un ulteriore tentativo migliorando il risultato.

Durante l'addestramento possiamo osservare i valori della `loss` ed eventualmente variare i `parametri` della rete per migliorare le prestazioni. In questo esempio i parametri sono pochi.

In [5]:
# Addestramento modello (training)
model.fit(xs, ys, epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 93ms/step - loss: 44.9357
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 35.7278
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 28.4758
Epoch 4/500
1/1 [==============================] - 0s 2ms/step - loss: 22.7627
Epoch 5/500
1/1 [==============================] - 0s 1ms/step - loss: 18.2605
Epoch 6/500
1/1 [==============================] - 0s 2ms/step - loss: 14.7112
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - loss: 11.9117
Epoch 8/500
1/1 [==============================] - 0s 2ms/step - loss: 9.7022
Epoch 9/500
1/1 [==============================] - 0s 2ms/step - loss: 7.9571
Epoch 10/500
1/1 [==============================] - 0s 14ms/step - loss: 6.5775
Epoch 11/500
1/1 [==============================] - 0s 2ms/step - loss: 5.4856
Epoch 12/500
1/1 [==============================] - 0s 4ms/step - loss: 4.6201
Epoch 13/500
1/1 [==============================] - 

2024-11-11 15:31:56.763299: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 39/500
1/1 [==============================] - 0s 2ms/step - loss: 0.8880
Epoch 40/500
1/1 [==============================] - 0s 1ms/step - loss: 0.8688
Epoch 41/500
1/1 [==============================] - 0s 1ms/step - loss: 0.8503
Epoch 42/500
1/1 [==============================] - 0s 2ms/step - loss: 0.8322
Epoch 43/500
1/1 [==============================] - 0s 2ms/step - loss: 0.8147
Epoch 44/500
1/1 [==============================] - 0s 1ms/step - loss: 0.7976
Epoch 45/500
1/1 [==============================] - 0s 2ms/step - loss: 0.7810
Epoch 46/500
1/1 [==============================] - 0s 2ms/step - loss: 0.7647
Epoch 47/500
1/1 [==============================] - 0s 1ms/step - loss: 0.7488
Epoch 48/500
1/1 [==============================] - 0s 1ms/step - loss: 0.7333
Epoch 49/500
1/1 [==============================] - 0s 1ms/step - loss: 0.7181
Epoch 50/500
1/1 [==============================] - 0s 1ms/step - loss: 0.7033
Epoch 51/500
1/1 [==============================] - 

Ora che abbiamo addestrato il modello, facendo in modo che imparasse la relazione tra `x` e `y` e valutando la `loss`, possiamo testarlo su un nuovo valore di `x` che la rete non ha mai "visto".
Se come valore di test poniamo `x=10` quale sarà il corrispodente valore di `y`?
Per prevedere il valore di `y` corrispondente al nuovo valore di `x`in questione usiamo il metodo [`model.predict()`](https://keras.io/api/models/model_training_apis/#predict-method).

In [6]:
# Previsione (dato di test)
print(f"Previsione : {model.predict(np.array([10.0]), verbose=0).item():.5f}")

Previsione : 18.97707


Il valore trovato dovrebbe corrispondere a `19` secondo le nostre aspettative (`y=2x-1`).
Come mai la rete ottiene un valore leggermente più basso?

I motivi sono due: 
* Le reti neurali si affidano al calcolo delle probabilità perciò il modello ci dirà, attraverso il valore calcolato nella fase di test con il metodo `predict`, che c'è una probabilità elevata che la relazione tra `x` e `y` sia `y=2x-1`.
* Pochi dati: abbiamo utilizzato solo 6 valori (pattern) di `x` e 6 valori corrispondenti di `y` nella fase di training. Con pochi pattern può capitare di ottenere risultati imprecisi per _underfitting_.

In generale con le reti neurali dobbiamo abituarci ad ottenere dati probabilistici, non dati certi.